In [ ]:
audiomentations_file = None
params_file = None
output_folder = None

In [1]:
# common libs
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import json

# audio
import librosa
import soundfile
from IPython.display import Audio
from audiomentations import *

In [ ]:
with open(params_file) as f:
    params = json.load(f)
num_sims = int(params['num_sims'])
num_time_steps = int(params['num_time_steps'])
num_agents = int(params['num_agents'])
num_seconds = int(params['num_seconds'])
step_std = float(params['step_std'])
inner_circle_radius = float(params['inner_circle_radius'])
sample_rate = int(params['sample_rate'])
folder = str(params['folder_with_birds'])

In [3]:
species = os.listdir(folder)
audio_dict = dict()
for _ in species:
    subfolder = os.path.join(folder, _)
    try:
        audio_files = [f for f in os.listdir(subfolder) if f.endswith('.ogg')]
        audio_dict[_] = audio_files
    except NotADirectoryError:
        pass
species_labels = {sp: idx for idx, sp in enumerate(species, start=1)}

In [4]:
# this forces the labeling to be alphabetical
species_labels_list = sorted(list(species_labels.keys()))
species_labels = {species_labels_list[i]:i+1 for i in range(len(species_labels_list))}
species_labels

{'carcha1': 1,
 'comior1': 2,
 'eucdov': 3,
 'fotdro5': 4,
 'perfal': 5,
 'purher1': 6,
 'ratcis1': 7,
 'saffin': 8,
 'tafpri1': 9,
 'whtkin2': 10}

## Define the Bird class

In [5]:
class Bird:
    def __init__(self, 
                 audio_path,
                 individual_label,
                 species_label,
                 step_std=0.05, 
                 call_length=5.,
                 sr=32000,
                 inner_circle_radius=0.25,
                 outer_circle_radius=1.,
                 ):
        '''Initialize a bird with its audio file and labels.'''
        self._step_std = step_std
        self._audio_path = audio_path
        self._sr = sr
        self._individual_label = individual_label
        self._species_label = species_label
        # lists for tracking
        self._individual_history = []
        self._species_history = []
        # set up radii
        self._outer_circle_radius = outer_circle_radius
        self._inner_circle_radius = inner_circle_radius
        # set up position
        angles = np.random.uniform(0, 2*np.pi)
        radii = np.sqrt(np.random.uniform(0, 1))
        self._position = np.array([radii * np.cos(angles), radii * np.sin(angles)])
        # store duration of audio file
        self._duration = librosa.get_duration(path=self._audio_path, sr=self._sr)
        # current point in recording
        self._call_length = call_length
        self._timestamp = np.random.uniform(0, self._duration - self._call_length)

    # Make a potential move each time step
    def step(self):
        step = np.random.normal(0, self._step_std, 2)
        # Move if not leaving outer circle
        if np.linalg.norm(self._position + step) <= self._outer_circle_radius:
            self._position += step
        # Move opposite if still inside outer circle
        elif np.linalg.norm(self._position - step) <= self._outer_circle_radius:
            self._position -= step

    # Record vocalization if inside inner circle
    def record(self, num_seconds=None):
        '''Make and record a vocalization if inside inner circle.'''
        if np.linalg.norm(self._position) <= self._inner_circle_radius:
            self._individual_history.append(self._individual_label)
            self._species_history.append(self._species_label)
            # Play audio
            voice, sr = self.vocalize(num_seconds=num_seconds)
            return voice, sr
        else:
            self._individual_history.append(0)
            self._species_history.append(0)
            return None, None

    # Make the vocalization
    def vocalize(self, num_seconds=None, sr=None):
        '''Load a segment of the audio file for vocalization.'''
        # Can specify how long to vocalize, otherwise use default
        if num_seconds is None:
            num_seconds = self._call_length
        # Ensure we don't exceed the audio duration
        if self._timestamp + num_seconds > self._duration:
            self._timestamp = np.random.uniform(0, self._duration - num_seconds)
        # Load the audio file and return the segment
        sr = sr if sr is not None else self._sr
        voice, sr = librosa.load(self._audio_path, sr=sr, 
                                 offset=self._timestamp, duration=num_seconds)
        # Update the timestamp for the next call
        self._timestamp += num_seconds
        # Return the audio data and sample rate
        return voice, sr

### Augment the sounds

In [ ]:
# for using papermill to run on terminal
# i.e., papermill -p output_folder [yours] -p agent_based_file [yours] -p audiomentations_file [yours] sim-soundscapes.ipynb [yours]
# process data in json file

with open(audiomentations_file) as f:
    params = json.load(f)

# this parameter is deprecated for the agent-based soundscape
num_audiomentations = int(params['num_audiomentations'])

# AddGaussianNoise()
min_amplitude_gaussian = float(params['min_amplitude_gaussian'])
max_amplitude_gaussian = float(params['max_amplitude_gaussian'])
p_gaussian = float(params['p_gaussian'])

# TimeStretch()
min_rate_timestretch = float(params['min_rate_timestretch'])
max_rate_timestretch = float(params['max_rate_timestretch'])
p_timestretch = float(params['p_timestretch'])

# PitchShift()
min_semitones_pitchshift = int(params['min_semitones_pitchshift'])
max_semitones_pitchshift = int(params['max_semitones_pitchshift'])
p_pitchshift = float(params['p_pitchshift'])

# Shift()
min_shift_shift = float(params['min_shift_shift'])
max_shift_shift = float(params['max_shift_shift'])
p_shift = float(params['p_shift'])

# Gain()
min_gain_db_gain = float(params['min_gain_db_gain'])
max_gain_db_gain = float(params['max_gain_db_gain'])
p_gain = float(params['p_gain'])


In [ ]:
# Define the augmentation pipeline

def augmenter(audio,
              sr,
              min_amplitude_gaussian,
              max_amplitude_gaussian,
              p_gaussian,
              min_rate_timestretch,
              max_rate_timestretch,
              p_timestretch,
              min_semitones_pitchshift,
              max_semitones_pitchshift,
              p_pitchshift,
              min_shift_shift,
              max_shift_shift,
              p_shift,
              min_gain_in_db_gain,
              max_gain_in_db_gain,
              p_gain,
             ):
    
    augment = Compose([
        
        # Add light ambient Gaussian noise with a 50% probability
        # min_amplitude=0.001 (very quiet) to max_amplitude=0.01 (noticeable but usually not overwhelming)
        AddGaussianNoise(min_amplitude=min_amplitude_gaussian, 
                         max_amplitude=max_amplitude_gaussian, 
                         p=p_gaussian),
    
        # Stretch the duration of the sound slightly (speed up or slow down) with 20% probability
        # rate 0.8 (slower) to 1.25 (faster)
        TimeStretch(min_rate=min_rate_timestretch, 
                    max_rate=max_rate_timestretch, 
                    p=p_timestretch),
    
        # Shift the pitch slightly up or down by a few semi-tones with 60% probability
        # -4 semi-tones to +4 semi-tones
        PitchShift(min_semitones=min_semitones_pitchshift, 
                   max_semitones=max_semitones_pitchshift, 
                   p=p_pitchshift),
    
        # Shift the time axis of the sound slightly with 70% probability
        # shift_ms can be positive or negative (forwards or backwards in time within the clip)
        Shift(min_shift=min_shift_shift, 
              max_shift=max_shift_shift, 
              p=p_shift),
        
        # Adjust the volume (gain) up or down slightly with 40% probability
        # min_gain_in_db=-6 (quieter) to max_gain_in_db=6 (louder)
        Gain(min_gain_db=min_gain_db_gain, 
             max_gain_db=max_gain_db_gain, 
             p=p_gain), 
        
    ])

    return augment(samples=audio, sample_rate=sr)

## Bird-based simulation

#### Many runs

In [8]:
n = 0
while n < num_sims:

    try:
        print(n)
        # Create a list of Bird agents
        birds = []
        for i in range(num_agents):
            spec = random.choice(list(audio_dict.keys()))
            audi = random.choice(audio_dict[spec])
            bird = Bird(
                step_std=step_std,
                audio_path=f"{folder}/{spec}/{audi}",
                individual_label=i+1,
                species_label=species_labels[spec],
                inner_circle_radius=inner_circle_radius,
                call_length=num_seconds,
                sr=sample_rate,
            )
            birds.append(bird)
    
        # Run the simulation for num_steps
        full_recording = []
        hot_ones = np.zeros((num_time_steps, max(species_labels.values()) + 1), dtype=int)
        for _ in range(num_time_steps):
            voices = []
            obs_spec = []
            for bird in birds:
                bird.step()
                voice, sr = bird.record(num_seconds=num_seconds)
                if voice is not None:
                    voices.append(voice)
                    obs_spec.append(bird._species_label)
            voices = np.array(voices) if len(voices) > 0 else np.zeros((1, sample_rate * num_seconds))
            hot_ones[_, obs_spec] = 1
            full_recording.append(np.sum(voices,axis=0))
        one_recording = np.concatenate(full_recording) # this can fail if time is too short
        audio2 = augmenter(one_recording, 
                   sample_rate,
                   min_amplitude_gaussian,
                   max_amplitude_gaussian,
                   p_gaussian,
                   min_rate_timestretch,
                   max_rate_timestretch,
                   p_timestretch,
                   min_semitones_pitchshift,
                   max_semitones_pitchshift,
                   p_pitchshift,
                   min_shift_shift,
                   max_shift_shift,
                   p_shift,
                   min_gain_db_gain,
                   max_gain_db_gain,
                   p_gain,
                  )
        augmented_recording = audio2
        hot_ones[np.argwhere(hot_ones.sum(axis=1) == 0), 0] = 1 # impute for silence
        soundfile.write(f"{output_folder}/simulated_audio_agents{num_agents}_radius{inner_circle_radius}_{n}.ogg", 
                        audio2, 
                        sample_rate, 
                        format='OGG', 
                        subtype='VORBIS')
        np.savetxt(f"{output_folder}/simulated_one_hot_agents{num_agents}_radius{inner_circle_radius}_{n}.csv", 
                   hot_ones,
                   delimiter=",",
                   fmt='%d',
                   )
        n += 1

    except:
        pass

0


/nfs/turbo/umor-sethtem/hoodriver/lib/python3.12/site-packages/audiomentations/core/transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


5
1
5
2
5
3
5
4
5
